## 임베딩 & 검색

**Chroma db 검색**
| 방식                              | 특징                                   |
| ------------------------------- | ------------------------------------ |
| `similarity_search`             | 가장 유사한 문서만 반환 (내용 중복 가능성 높음)         |
| `max_marginal_relevance_search` | 유사성 + 다양성 고려 (중복 줄이고 다양한 내용 반환)      |
| `MultiQueryRetriever`           | LLM이 다양한 형태로 질문을 변형 → 여러 관점에서 포괄적 검색 |

1. Similarity Search (유사도 검색) : similarity_search(question, k=2)
- 기준: 질문과 가장 유사한 문서들을 찾음
- 특징: 코사인 유사도 등으로 순수하게 유사도만 고려
- 결과: 비슷한 내용의 문서들이 함께 선택될 수 있음
- 장점: 빠르고 정확한 유사도 기반 검색
- 단점: 중복된 정보가 많을 수 있음
- 특정 주제에 대한 정확한 정보가 필요할 때 사용

2. Maximum Marginal Relevance (MMR) : max_marginal_relevance_search(question, k=2, fetch_k=3)

- 기준: 유사성 + 다양성을 모두 고려
- 과정:
    - fetch_k=3개의 유사한 문서를 먼저 찾음
    - 그 중에서 서로 다른 정보를 담은 k=2개를 선택
- 특징: 중복을 피하고 다양한 관점의 정보 제공
- 장점: 더 포괄적이고 다양한 정보
- 단점: 계산이 더 복잡함
- 주제에 대한 다양한 관점과 포괄적인 정보가 필요할 때 사용

3. MultiQueryRetriever
- chromadb.as_retriever() → 벡터스토어를 retriever로 변환
- MultiQueryRetriever.from_llm() → 하나의 질문을 여러 변형 쿼리로 검색 (누락 줄임)
- 따라서 일반 검색보다 더 다양한 문서를 포착할 수 있음

In [ ]:
# !pip install chromadb

In [1]:
# from openai import OpenAI
import os
from dotenv import load_dotenv

# 환경 변수 로드 및 OpenAI 클라이언트 설정
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 1. OpenAI 임베딩 모델 초기화
embedding = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)  # ... code here

/opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


```Markdown
texts = [
    # 생성형 AI 개념 설명
    "생성형 AI는 기존 데이터를 학습하여 새로운 콘텐츠를 자동으로 생성하는 인공지능 기술이다.",
    
    # 생성형 AI의 정의 강조
    "생성형 인공지능이란, 사용자의 명령에 따라 텍스트, 이미지, 음성 등을 창조해내는 모델이다.",
    
    # 생성형 AI와 전통적 AI의 차이
    "전통적인 AI는 규칙 기반으로 작동하지만, 생성형 AI는 학습한 패턴을 기반으로 창의적인 출력을 생성한다.",
    
    # 생성형 AI 활용 예시
    "생성형 AI는 마케팅 콘텐츠 제작, 음악 생성, 코딩 보조 등 다양한 분야에 활용되고 있다.",
    
    # ChatGPT에 초점
    "ChatGPT는 생성형 AI 모델 중 하나로, 대화형 응답을 생성하는 데 최적화되어 있다.",
    
    # 이미지 생성 중심
    "이미지 생성 AI인 DALL·E는 텍스트 설명을 바탕으로 고해상도 이미지를 생성한다.",
    
    # 생성형 AI 시장 동향
    "2023년 이후 생성형 AI 시장은 급성장하고 있으며, 글로벌 기업들이 기술 확보에 나서고 있다.",
    
    # 사회적 영향
    "생성형 AI는 창작의 패러다임을 바꾸며, 윤리적 문제나 저작권 이슈도 함께 제기되고 있다.",
    
    # 대규모 언어 모델의 기반 기술
    "생성형 AI는 트랜스포머 기반의 대규모 언어 모델(LLM)을 핵심 기술로 삼고 있다.",
    
    # 비슷하지만 살짝 다른 주제 (비생성형 AI)
    "분류 모델, 추천 시스템 등은 생성형 AI가 아닌 전통적인 분석형 AI로 분류된다."
]
```

In [3]:
# 2. 검색할 텍스트 데이터 준비 (5개의 생성형 AI 관련 문서)
texts = [
    "생성형 AI는 기존 데이터를 학습하여 새로운 콘텐츠를 자동으로 생성하는 인공지능 기술이다.",
    "생성형 인공지능이란, 사용자의 명령에 따라 텍스트, 이미지, 음성 등을 창조해내는 모델이다.",
    "전통적인 AI는 규칙 기반으로 작동하지만, 생성형 AI는 학습한 패턴을 기반으로 창의적인 출력을 생성한다.",
    "생성형 AI는 마케팅 콘텐츠 제작, 음악 생성, 코딩 보조 등 다양한 분야에 활용되고 있다.",
    "ChatGPT는 생성형 AI 모델 중 하나로, 대화형 응답을 생성하는 데 최적화되어 있다.",
    "이미지 생성 AI인 DALL·E는 텍스트 설명을 바탕으로 고해상도 이미지를 생성한다.",
    "2023년 이후 생성형 AI 시장은 급성장하고 있으며, 글로벌 기업들이 기술 확보에 나서고 있다.",
    "생성형 AI는 창작의 패러다임을 바꾸며, 윤리적 문제나 저작권 이슈도 함께 제기되고 있다.",
    "생성형 AI는 트랜스포머 기반의 대규모 언어 모델(LLM)을 핵심 기술로 삼고 있다.",
    "분류 모델, 추천 시스템 등은 생성형 AI가 아닌 전통적인 분석형 AI로 분류된다."
]


In [4]:
# 3. Chroma 벡터 데이터베이스 생성 (텍스트를 임베딩으로 변환하여 저장)
chromadb = Chroma.from_texts(texts=texts, embedding=embedding)  # ... code here

In [5]:
question = "생성형 AI에 대해 언급된 내용은?"
# question = "생성형 AI 핵심 기능은?"

In [6]:
# 4. MMR 검색 : k=2, fetech_k=3
docs_mmr = chromadb.max_marginal_relevance_search(query=question, k=3, fetch_k=5)  # ... code here
print(docs_mmr[0].page_content[:100])
print('\n\n', docs_mmr[1].page_content[:100])
print('\n\n', docs_mmr[2].page_content[:100])

생성형 AI는 마케팅 콘텐츠 제작, 음악 생성, 코딩 보조 등 다양한 분야에 활용되고 있다.


 생성형 인공지능이란, 사용자의 명령에 따라 텍스트, 이미지, 음성 등을 창조해내는 모델이다.


 생성형 AI는 트랜스포머 기반의 대규모 언어 모델(LLM)을 핵심 기술로 삼고 있다.


In [8]:
# 5. 유사도 검색 (Similarity Search) - 가장 유사한 문서 2개 반환
docs_ss = chromadb.similarity_search(query=question,k=3)
print(docs_ss[0].page_content[:100])
print('\n', docs_ss[1].page_content[:100])
print('\n', docs_ss[2].page_content[:100])

생성형 AI는 마케팅 콘텐츠 제작, 음악 생성, 코딩 보조 등 다양한 분야에 활용되고 있다.

 생성형 AI는 기존 데이터를 학습하여 새로운 콘텐츠를 자동으로 생성하는 인공지능 기술이다.

 생성형 인공지능이란, 사용자의 명령에 따라 텍스트, 이미지, 음성 등을 창조해내는 모델이다.


In [9]:
# 6. 최대 한계 관련성 검색 (MMR) - 유사성과 다양성을 고려한 2개 반환
docs_mmr1 = chromadb.max_marginal_relevance_search(query=question,k=3)
print(docs_mmr1[0].page_content[:100])
print('\n',docs_mmr1[1].page_content[:100])
print('\n',docs_mmr1[2].page_content[:100])

생성형 AI는 마케팅 콘텐츠 제작, 음악 생성, 코딩 보조 등 다양한 분야에 활용되고 있다.

 분류 모델, 추천 시스템 등은 생성형 AI가 아닌 전통적인 분석형 AI로 분류된다.

 이미지 생성 AI인 DALL·E는 텍스트 설명을 바탕으로 고해상도 이미지를 생성한다.


In [10]:
# 7. MultiQueryRetriever 검색
# LLM 초기화

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=api_key)

# MultiQueryRetriever 구성
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=chromadb.as_retriever(),
    llm=llm,
)

# 검색 수행
docs_multi = retriever_from_llm.invoke({"query": question})
print("[MultiQueryRetriever 결과]")
for i, doc in enumerate(docs_multi):
    print(f"[{i+1}] {doc.page_content[:100]}")

[MultiQueryRetriever 결과]
[1] 생성형 인공지능이란, 사용자의 명령에 따라 텍스트, 이미지, 음성 등을 창조해내는 모델이다.
[2] 생성형 AI는 마케팅 콘텐츠 제작, 음악 생성, 코딩 보조 등 다양한 분야에 활용되고 있다.
[3] 생성형 AI는 기존 데이터를 학습하여 새로운 콘텐츠를 자동으로 생성하는 인공지능 기술이다.
[4] 생성형 AI는 트랜스포머 기반의 대규모 언어 모델(LLM)을 핵심 기술로 삼고 있다.
[5] 생성형 AI는 창작의 패러다임을 바꾸며, 윤리적 문제나 저작권 이슈도 함께 제기되고 있다.


In [12]:
from langchain.chains import RetrievalQA       # 질의응답

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retreiver = chromadb.as_retriever(search_kwargs = {"k": 3}),
    return_source_documents = True,
)

result = qa_chain.invoke({"query": question})
print(result)

for doc in result['source_documents']:
    print(doc.page_content)

ValidationError: 2 validation errors for RetrievalQA
retriever
  Field required [type=missing, input_value={'combine_documents_chain...source_documents': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
retreiver
  Extra inputs are not permitted [type=extra_forbidden, input_value=VectorStoreRetriever(tags... search_kwargs={'k': 3}), input_type=VectorStoreRetriever]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden